In [5]:
# Import required packages 
import os
import glob
import cv2
import numpy as np
import pandas as pd
import random

# Read the training data file with error handling for bad lines
data = pd.read_excel("D:\\updated_final_book_1.xlsx")


# Display the first few rows
print(data.head())
print(list(data.columns))
data.info()


def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Check dimensions of the subset 
print(data.shape)
data['HighRes_Drive_Path'] = data['HighRes_Drive_Path'].apply(lambda x: convert_path(x.strip()))


        Date Flight  Total_flights Start_time End_time    Drone Pilot  \
0 2023-08-07      A              1        NaN      NaN  Pink_M3    AD   
1 2023-08-07      A              1        NaN      NaN  Pink_M3    AD   
2 2023-08-07      A              1        NaN      NaN  Pink_M3    AD   
3 2023-08-07      A              1        NaN      NaN  Pink_M3    AD   
4 2023-08-07      A              1        NaN      NaN  Pink_M3    AD   

  Location     Video     Pack  ... light quality habitat overall_quality  \
0     GCP1  DJI_0376  Mudhale  ...          good    good            good   
1     GCP1  DJI_0377  Mudhale  ...         good     good            okay   
2     GCP1  DJI_0378  Mudhale  ...          good    good            okay   
3     GCP1  DJI_0379  Mudhale  ...          good    good            okay   
4     GCP1  DJI_0380  Mudhale  ...          good    good            okay   

   shortlisted  frames_extracted                      unique_id  \
0          yes               yes  070

In [6]:
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import os

def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Define a function to trim the video
def trim_video(row):
    video_path = convert_path(row['HighRes_Drive_Path'].strip())  # Normalize and strip any leading/trailing whitespace
    if video_path.startswith("\\"):
        video_path = video_path[1:]  # Remove leading backslash if present
    start_time = row['start']
    end_time = row['end']
    unique_id = row['unique_id']
    
    try:
        # Load the video
        with VideoFileClip(video_path) as video:
            if start_time == "start" and pd.notnull(end_time) and end_time != "end":
                end_time = float(end_time)
                trimmed_video = video.subclip(0, end_time)
            elif end_time == "end" and pd.notnull(start_time) and start_time != "start":
                start_time = float(start_time)
                trimmed_video = video.subclip(start_time, video.duration)
            elif start_time == "start" and end_time == "end":
                trimmed_video = video
            elif pd.notnull(start_time) and pd.notnull(end_time) and start_time != "start" and end_time != "end":
                start_time = float(start_time)
                end_time = float(end_time)
                trimmed_video = video.subclip(start_time, end_time)
            else:
                raise ValueError(f"Invalid start or end time for unique ID: {unique_id}")

            # Define the output directory and filename
            output_folder = os.path.join(output_directory, unique_id)
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{unique_id}_P01.mp4"
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the trimmed video with the same quality as the original
            trimmed_video.write_videofile(output_path, codec='libx264', preset='ultrafast', audio_codec='aac', temp_audiofile='temp-audio.m4a', remove_temp=True)
            print(f"Successfully processed: {output_filename}")

    except Exception as e:
        print(f"Error processing {unique_id}: {e}")

# Function to process videos by unique IDs
def process_videos_by_ids(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Debug statement to check the unique ID
        print(f"Processing unique ID: {unique_id}")

        # Find the row corresponding to the unique ID
        selected_row = data[data['unique_id'].astype(str).str.strip() == unique_id]
        
        if not selected_row.empty:
            video_path = selected_row['HighRes_Drive_Path'].values[0]
            print(f"Found video path: {video_path}")
            selected_row.apply(trim_video, axis=1)
        else:
            print(f"No video found with the unique ID: {unique_id}")

import random
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def extract_frames(video_path, output_folder):
    video = VideoFileClip(video_path)
    duration = int(video.duration)

    for sec in range(duration):
        frame_time = sec + random.random()  # Random time within each second
        frame = video.get_frame(frame_time)
        frame_filename = f"{os.path.basename(video_path).split('.')[0]}_{sec+1}_{int(frame_time*1000)}.png"
        frame_path = os.path.join(output_folder, frame_filename)
        video.save_frame(frame_path, t=frame_time)
        print(f"Extracted frame at {frame_time:.2f}s to {frame_path}")

# Example usage
def process_extracted_videos(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Define the output folder
        output_folder = os.path.join(output_directory, unique_id)
        video_filename = f"{unique_id}_P01.mp4"
        video_path = os.path.join(output_folder, video_filename)
        
        if os.path.exists(video_path):
            print(f"Processing extracted frames for video: {video_path}")
            extract_frames(video_path, output_folder)
        else:
            print(f"No trimmed video found for unique ID: {unique_id}")



In [7]:

# Ensure the output directory exists
output_directory = "D:\\wolf_drone_data\\cut_videos"

# Define your list of unique IDs (copy multiple IDs) 
unique_ids = ['0908_005_PM3_Turn_SH_DJI_0408']


In [8]:
# # Process/ trim the videos by unique ID
process_videos_by_ids(unique_ids)

Processing unique ID: 0908_005_PM3_Turn_SH_DJI_0408
Found video path: D:\wolf_drone_data\Aug_2023\Drone_data\Mavic3\Pink_M3\Mudhale\090823\DJI_0408.MP4
Moviepy - Building video D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01.mp4.
Moviepy - Writing video D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01.mp4



Moviepy - Done !
Moviepy - video ready D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01.mp4
Successfully processed: 0908_005_PM3_Turn_SH_DJI_0408_P01.mp4


In [9]:
# Process the extracted frames for the trimmed videos
process_extracted_videos(unique_ids)

Processing extracted frames for video: D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01.mp4
Extracted frame at 0.70s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_1_702.png
Extracted frame at 1.08s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_2_1083.png
Extracted frame at 2.96s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_3_2964.png
Extracted frame at 3.62s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_4_3620.png
Extracted frame at 4.12s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_5_4121.png
Extracted frame at 5.09s to D:\wolf_drone_data\cut_videos\0908_005_PM3_Turn_SH_DJI_0408\0908_005_PM3_Turn_SH_DJI_0408_P01_6_5091.png
Extracted frame at 6.92s to D:\wolf_drone_data\cut_videos\0908_005